# Importing Libraries

In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                             \                                       failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='2_73_pypy|4_73_pypy|3_73_pypy|1_73_pypy|0_73_pypy']
  - geopy -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']
  - rsa -> python[version='2.7.*|3.4.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                    -failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='1_73_pypy|2_73_pypy|4_73_pypy|3_73_pypy|0_73_pypy']
  - rsa -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']

Your python: python=3.7

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, t

Libraries imported.


In [2]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


# Downloading Data in Dataframe

In [29]:
from bs4 import BeautifulSoup

In [30]:
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

source= requests.get(url).text
soup= BeautifulSoup(source)
table_data=soup.find('div',
class_='mw-parser-output')
table=table_data.table.tbody

columns= ['Postal Code', 'Borough', 'Neighborhood']
data=dict({key:[]*len(columns) for key in columns})

for row in table.find_all('tr'):
    for i, column in zip(row.find_all('td'),columns):
        i=i.text
        i=i.replace('\n',
'')
        data[column].append(i)
        
df= pd.DataFrame.from_dict(data=data)[columns]
print(df.shape)
df.head()

(180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Dropping Not Assigned Boroughs

In [32]:
df= df[df['Borough'] != 'Not assigned'].reset_index(drop = True)
print('After dropping rows where borough is "Not Assigned", Shape is: ',df.shape)
print('Number of rows where Neighborhood is "Not Assigned" but borough has value: ',
     df[df['Neighborhood'] == 'Not assigned'].shape[0])

After dropping rows where borough is "Not Assigned", Shape is:  (103, 3)
Number of rows where Neighborhood is "Not Assigned" but borough has value:  0


In [33]:
p, b, n =[], [], []

for postcode, borough, neigh in zip(df['Postal Code'], df['Borough'], df['Neighborhood']):
    p.append(postcode)
    b.append(borough)
    if neigh == 'Not assigned':
        n.append(borough)
    else:
        n.append(neigh)

df=pd.DataFrame({'Postal Code':p,'Borough':b, 'Neighborhood':n})[columns]
print(df.shape)
df.head()

(103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# <font size="4"> 77 rows have been dropped because there was no assigned boroughs for them.</font>

# <font size="5"> Original Data had 180 rows and now the data is having 103 rows.</font>

In [35]:
postcodes=df['Postal Code'].values
boroughs =df['Borough'].values
neighs=df['Neighborhood'].values

dic=dict({(key1, key2): [] for key1, key2 in zip(postcodes, boroughs)})
print('Number of keys in the dictionary are: ',len(dic.keys()))

for postcode, borough, neigh in zip(postcodes, boroughs, neighs):
    key=(postcode, borough)
    dic[key].append(neigh)

df=pd.DataFrame(columns=['Postal Code','Borough','Neighborhood'])
for key, value in dic.items():
    postcode, borough, neig=key[0],key[1], value
    neig=', '.join(neig)
    df=df.append({'Postal Code': postcode,
                 'Borough': borough,
                 'Neighborhood': neig}, ignore_index=True)

print('Shape of final data is: ',df.shape)
df.head(10)

Number of keys in the dictionary are:  103
Shape of final data is:  (103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# Latitude and Longitude of Each Neighbourhood

In [18]:
pip install geocoder

     |████████████████████████████████| 98 kB 8.6 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [19]:
import geocoder # import geocoder

In [21]:
pip install pgeocode

Note: you may need to restart the kernel to use updated packages.


# Downlaoding the GeoSpatial Data by downloading coordinates of postal codes

In [36]:
!wget http://cocl.us/Geospatial_data

--2020-12-19 14:25:29--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.63.96.176, 169.63.96.194
Connecting to cocl.us (cocl.us)|169.63.96.176|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-12-19 14:25:29--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.63.96.176|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-12-19 14:25:30--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-12-19 14:25:30--  https://ibm.box.com/public/static/9afzr8

In [37]:
try:
    df['Latitude']= lats
    df['Longitude']= lons
except:
    latlon=pd.read_csv('Geospatial_data')
    df=pd.merge(df, latlon, how='inner', on= 'Postal Code')

print(df.shape)
df.head(10)

(103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
